In [2]:
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration, BitsAndBytesConfig, Blip2ForImageTextRetrieval
import torch
import pandas as pd

In [1]:
import transformers
print(transformers.__version__)  # should now show 4.56.1
print(transformers.__file__)

C:\Users\Sandra\PycharmProjects\ProductLifetime\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.57.0.dev0
C:\Users\Sandra\PycharmProjects\ProductLifetime\venv\Lib\site-packages\transformers\__init__.py


In [3]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [20]:
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")

model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-flan-t5-xl"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.42it/s]


In [21]:
csv = "powerpoint_data.csv"
df = pd.read_csv(csv)

pairs = []
for _, row in df.iterrows():
    pairs.append((row['fixed title'],row['fixed image dir'], row['fixed body']))
    pairs.append((row['waiting title'], row["waiting image dir"], row["waiting body"]))
    pairs.append((row['not fixed title'], row["not fixed image dir"], row["not fixed body"]))

titles  = [p[0] for p in pairs]
image_paths = [p[1] for p in pairs]
all_texts   = [p[2] for p in pairs]

In [22]:
batch_size = 16
generated_texts = []

for i in range(0, len(image_paths), batch_size):
    batch_paths = image_paths[i:i+batch_size]
    images = [Image.open(p) for p in batch_paths]
    
    inputs = processor(images=images, return_tensors="pt")
    inputs = {k: v.to(device, dtype=torch.float16) for k, v in inputs.items()}
    
    generated_ids = model.generate(**inputs, max_new_tokens=50)
    batch_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
    
    generated_texts.extend([t.strip() for t in batch_texts])


In [23]:
print(generated_texts[1])

a chair is sitting on top of a table in a plastic bag


In [29]:
processor = Blip2Processor.from_pretrained(
    "Salesforce/blip2-flan-t5-xl",
    cache_dir="C:/Users/Sandra/PycharmProjects/ProductLifetime/hf_cache"
)

model2 = Blip2ForImageTextRetrieval.from_pretrained(
    "Salesforce/blip2-flan-t5-xl",
    cache_dir="C:/Users/Sandra/PycharmProjects/ProductLifetime/hf_cache"
).to(device)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]C:\Users\Sandra\PycharmProjects\ProductLifetime\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sandra\PycharmProjects\ProductLifetime\hf_cache\models--Salesforce--blip2-flan-t5-xl. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkp

In [34]:
def compute_alignment(image_path, text, processor, model, device="cpu"):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, text=text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits_per_image, dim=1)  # shape [1, 2]
    return probs[0, 1].item()

In [38]:
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl-coco")
model_itm = Blip2ForImageTextRetrieval.from_pretrained("Salesforce/blip2-flan-t5-xl-coco").to(device)

prob = compute_alignment(image_paths[0], generated_texts[0], processor, model_itm, device=device)
print(f"Alignment probability: {prob:.2%}")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]C:\Users\Sandra\PycharmProjects\ProductLifetime\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sandra\.cache\huggingface\hub\models--Salesforce--blip2-flan-t5-xl-coco. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 

AttributeError: 'Blip2QFormerLayer' object has no attribute 'intermediate'